In [1]:
%load_ext autoreload

In [5]:
%autoreload 2

from hydra import compose, initialize
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm
import h5py
import numpy as np
from UniversalUnconsciousness.data_utils import *
from UniversalUnconsciousness.plot_utils import *
from UniversalUnconsciousness.power_analysis import *

plt.style.use('UniversalUnconsciousness.sci_style')
load_font()

In [3]:
with initialize(version_base="1.3", config_path="../UniversalUnconsciousness/DeLASE_analysis/conf"):
    cfg = compose(config_name="config")

In [4]:
cfg.params.low_pass = None
cfg.params.high_pass = None

In [11]:
anesthetic_agent_list = cfg.plotting.anesthetic_agent_list
img_save_dir = cfg.plotting.img_save_dir
curve_colors = cfg.plotting.curve_colors
loc_roc_colors = cfg.plotting.loc_roc_colors
os.makedirs(img_save_dir, exist_ok=True)

In [8]:
agent_data = get_agent_data(cfg, use_individiual_areas=False, verbose=True)

  0%|          | 0/3 [00:00<?, ?it/s]

File found for session MrJones-Anesthesia-20160125-01: /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/noise_filter_info/propofolPuffTone/MrJones-Anesthesia-20160125-01__window_15__wakethresh_1__anesthesiathresh_1__electrodenum_1__stride_15.pkl
23 bad electrodes, 535 valid windows (out of 672 total windows)
File found for session MrJones-Anesthesia-20160210-01: /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/noise_filter_info/propofolPuffTone/MrJones-Anesthesia-20160210-01__window_15__wakethresh_1__anesthesiathresh_1__electrodenum_1__stride_15.pkl
18 bad electrodes, 395 valid windows (out of 650 total windows)
File found for session Mary-Anesthesia-20160818-02: /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/noise_filter_info/propofolPuffTone/Mary-Anesthesia-20160818-02__window_15__wakethresh_1__anesthesiathresh_1__electrodenum_1__stride_15.pkl
7 bad electrodes, 219 valid windows (out of 377 total windows)
File found for session MrJones-Anesthesia-20160121

Getting Grid Search Run List:   0%|          | 0/21 [00:00<?, ?it/s]

list exists! loading /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/grid_searches/propofolPuffTone/grid_search_run_lists/MrJones-Anesthesia-20160125-01_grid_set6_window_15...
*COMPLETE*: All results completed for MrJones-Anesthesia-20160125-01 - all!!
list exists! loading /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/grid_searches/propofolPuffTone/grid_search_run_lists/MrJones-Anesthesia-20160210-01_grid_set6_window_15...
*COMPLETE*: All results completed for MrJones-Anesthesia-20160210-01 - all!!
list exists! loading /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/grid_searches/propofolPuffTone/grid_search_run_lists/Mary-Anesthesia-20160818-02_grid_set6_window_15...
*COMPLETE*: All results completed for Mary-Anesthesia-20160818-02 - all!!
list exists! loading /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/grid_searches/propofolPuffTone/grid_search_run_lists/MrJones-Anesthesia-20160121-01_grid_set6_window_15...
*COMPLETE*: All results complet

  0%|          | 0/300 [00:00<?, ?it/s]

...

  0%|          | 0/1063 [00:00<?, ?it/s]

--------------------
SESSION = PEDRI_Dexmedetomidine_20220317
--------------------
Loading data for PEDRI_Dexmedetomidine_20220317 - all from /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/DeLASE_results/anesthesiaLvrOdd/delase_results/PEDRI_Dexmedetomidine_20220317/NOISE_FILTERED_15_1_1_1_stride_15/NOT_NORMED/SUBSAMPLE_1/NO_FILTER/WINDOW_15/GRID_RESULTS_n_delays_9_rank_900/STRIDE_15/all/NO_PCA


  0%|          | 0/1181 [00:00<?, ?it/s]

--------------------
SESSION = SPOCK_Dexmedetomidine_20210921
--------------------
Loading data for SPOCK_Dexmedetomidine_20210921 - all from /scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/DeLASE_results/anesthesiaLvrOdd/delase_results/SPOCK_Dexmedetomidine_20210921/NOISE_FILTERED_15_1_1_1_stride_15/NOT_NORMED/SUBSAMPLE_1/NO_FILTER/WINDOW_15/GRID_RESULTS_n_delays_7_rank_800/STRIDE_15/all/NO_PCA


  0%|          | 0/1138 [00:00<?, ?it/s]

In [9]:
area = 'all'
# save_dir = "/orcd/data/ekmiller/001/eisenaj/UniversalUnconsciousness/power_analysis"
# save_dir = "/scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/power_analysis"
save_dir = "/scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/power_analysis_v2"

# Band Analysis

In [12]:
top_percent = 0.1
for data_class, agent in tqdm(anesthetic_agent_list):
    cfg.params.data_class = data_class
    delase_results = agent_data[(data_class, agent)]['delase_results']
    agent_save_dir = os.path.join(save_dir, data_class, agent, area)
    os.makedirs(agent_save_dir, exist_ok=True)
    for session in tqdm(agent_data[(data_class, agent)]['session_list']):
        if not os.path.exists(os.path.join(agent_save_dir, f'{session}__top_percent_{top_percent}_power_analysis.pkl')):
            print(f"Performing power analysis for {session}...")
            freq_powers, freq_r2_scores = perform_power_analysis(delase_results, cfg, session, area, top_percent=top_percent, verbose=True)
            ret_dict = {'freq_powers': freq_powers, 'freq_r2_scores': freq_r2_scores}
            pd.to_pickle(ret_dict, os.path.join(agent_save_dir, f'{session}__top_percent_{top_percent}_power_analysis.pkl'))
        else:
            print(f"Power analysis for {session} already exists")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Performing power analysis for MrJones-Anesthesia-20160125-01...


  0%|          | 0/671 [00:00<?, ?it/s]

Performing power analysis for MrJones-Anesthesia-20160210-01...


...

  0%|          | 0/1063 [00:00<?, ?it/s]

Performing power analysis for PEDRI_Dexmedetomidine_20220317...


  0%|          | 0/1181 [00:00<?, ?it/s]

Performing power analysis for SPOCK_Dexmedetomidine_20210921...


  0%|          | 0/1138 [00:00<?, ?it/s]